# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.train.hyperdrive import RandomParameterSampling, choice, uniform
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal, BanditPolicy
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig, Experiment, Workspace, Environment

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
experiment_name = 'student-math-experiment-hyperdrive'

experiment=Experiment(ws, experiment_name)

In [4]:
from azureml.core import Dataset

# Load dataset from Azure ML workspace
df = Dataset.get_by_name(workspace=ws, name="student-math")

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

The model being used in this configuration is a machine learning model for hyperparameter tuning with early termination, leveraging Azure’s HyperDrive for optimization. The early termination policy is set with an `evaluation_interval` of 2, meaning the agent evaluates the model’s performance every two iterations, but with a delay of 6 iterations before the first evaluation, giving the model ample time to learn before performance checks begin. The slack factor of 0.1 allows for up to a 10% performance drop before considering early termination. Hyerparameters are selected using RandomParameterSampling, which chooses from a range of values for parameters like `n_estimators`, `max_depth`, and `min_samples_split` using a mix of discrete choices and continuous distributions. The estimator is based on the`SKLearn` framework, using a custom script (`train.py`) in a specified compute environment. The HyperDriveConfig manages the tuning process, specifying a maximum of 20 trials with a goal to maximize the primary metric, normalized_mean_absolute_error, while limiting concurrent runs to 1 to manage resource usage effectively. This configuration is designed to optimize the model while balancing performance checks and resource constraints.


In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, delay_evaluation=6)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "--n_estimators": choice(50, 100, 150),
    "--max_depth": choice(3, 5, 10, None),
    "--min_samples_split": uniform(0.1, 0.5),
})

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(
    source_directory=".",
    entry_script="train.py",
    compute_target="notebook-compute",
    environment=Environment.get(workspace=ws, name="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu"),
)

hyperdrive_run_config = HyperDriveConfig(
    estimator=estimator,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name="normalized_mean_absolute_error",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=1,
)

In [ ]:
#TODO: Submit your experiment
run = experiment.submit(workspace=ws, name='student-math-hyperdrive')

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

